In [1]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
from urllib.request import urlretrieve
from zipfile import ZipFile

Download files from https://download.geofabrik.de/:

In [11]:
ab_url = "https://download.geofabrik.de/north-america/canada/alberta-latest-free.shp.zip"
urlretrieve(ab_url, "../data/ab-osm.zip")

('../data/ab-osm.zip', <http.client.HTTPMessage at 0x731289b3bed0>)

In [12]:
with ZipFile("../data/ab-osm.zip", "r") as zObject:
    zObject.extractall(path="../data/raw/osm-data")

In [ ]:
osm = list()
p = Path("../data/raw/osm-data/")
for i in p.glob("*.shp"):
    df = gpd.read_file(i, engine="pyogrio")
    keyname = i.name.replace("gis_osm_", "").replace("_free_1.shp", "")
    df["key"] = keyname
    osm.append(df)

osm = pd.concat(osm)
osm = gpd.GeoDataFrame(osm)
osm.head()

In [ ]:
try:
    os.makedirs("../data/processed")
except FileExistsError:
    pass

osm.to_file("../data/processed/osm.gpkg", driver="GPKG")

In [ ]:
schema = pd.read_csv("../data/tables/osm-lulc.csv")
schema = schema.dropna(subset="lulc")
schema

In [ ]:
lulc_classes <- unique(schema$lulc)

for (lulc_class in lulc_classes) {
  keys <- schema |>
    filter(lulc == lulc_class) |>
    distinct(key) |>
    pluck("key")

  lulc_res <- list()

  for (k in keys) {
    values <- schema |>
      filter(lulc == lulc_class, key == k) |>
      distinct(value) |>
      pluck("value")

    data <- q |>
      add_osm_feature(key = k, value = values) |>
      osmdata_sf()
    data <- data$osm_polygons
    lulc_res[[k]] <- data
  }

  if (length(lulc_res) > 1) {
    lulc_res <- bind_rows(lulc_res)
  } else {
    lulc_res <- lulc_res[[1]]
  }
  osm[[lulc_class]] <- lulc_res
}

osm <- bind_rows(osm, .id = "lulc")
unique(osm$lulc)

In [14]:
lulc_classes = schema["lulc"].unique()
lulc_classes

array(['urban-low-density', 'commerical/residential/industrial',
       'developed-open-space', 'developed-construction', 'water',
       'snow_ice', 'rock_soil', 'forest', 'scrubland', 'grassland',
       'wetland', 'urban_open', 'urban_low', 'urban_construction',
       'urban_high', 'cropland'], dtype=object)

In [ ]:
for lulc_class in lulc_classes:
    keys = schema.loc[schema.lulc == lulc_class]
    keys = keys["key"].unique()
    
    lulc_res = list()
    
    for k in keys:
        values = schema.loc[(schema.lulc == lulc_class) & (schema.key == k)]
        

In [ ]:
with open("landcover-osm.yaml", "r") as f:
    schema = yaml.safe_load(f)

file_keys = ["landuse", "natural", "pofw", "pois", "traffic", "transport", "water",
             "buildings"]
geometries = list()

for f in file_keys:
    df = gpd.read_file(f"data/alberta-latest-free/gis_osm_{f}_a_free_1.shp")
    df["key"] = f
    geometries.append(df)

osm = pd.concat(geometries)

lc_types = list()

for lulc, tags in schema.items():
    for key, values in tags.items():
        if isinstance(values, str):
            values = [values]
        df = osm.loc[osm.fclass.isin(values)]
        df["lulc"] = lulc
        lc_types.append(df)

lc_types = pd.concat(lc_types)
lc_types.loc[lc_types.lulc == "water"].plot(column="fclass")

geom = lc_types.loc[lc_types.lulc == "water"]
geom = geom.to_json()
geo_j = folium.GeoJson(geom)
m = folium.Map(tiles="CartoDB Positron")
geo_j.add_to(m)
m